In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps
from collections import defaultdict
from time import sleep
from subprocess import Popen, PIPE, STDOUT
from tqdm.auto import tqdm
import graph_tool.all as gr

import subprocess

In [4]:
def get_pair(i):
    k = np.floor((1 + np.sqrt(1 + 8 * i)) / 2)
    m = i - k * (k - 1) / 2
    
    return int(k), int(m)


# def get_graph(i, n):
#     m = int(n * (n - 1) / 2)
#     edges = defaultdict(list)
    
#     for j in range(m):
#         if (i >> j) & 1 == 1:
#             u, v = get_pair(j)
#             edges[u].append(v)
#             edges[v].append(u)
    
#     return edges


def get_graph(code):
    parse = list(map(lambda x: f'{ord(x) - 63:06b}', code))
    n = int(parse[0], 2)
    m = int(n * (n - 1) / 2)
    parse = ''.join(parse[1:])[:m]
    
    edges = defaultdict(list)
    
    num = 0
    for i, byte in enumerate(parse):
        if byte == '1':
            num += 1
            u, v = get_pair(i)
            edges[u].append(v)
    
    return num, edges
    

def draw_graph(n, edges):
    graph = gr.Graph(directed=False)
    
    vertexes = list(graph.add_vertex(n))
    for u in edges:
        for v in edges[u]:
            graph.add_edge(vertexes[u], vertexes[v])
            
    gr.graph_draw(graph)
    
    
    
def get_output(input_data):
    p = Popen(['./build/3coloring'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
    return p.communicate(input=input_data.encode())[0].decode()


def test_graphs(n):    
    error = False
    
    with open(f'data/graph{n}c.g6', 'r') as data:
        for i, code in tqdm(enumerate(data)):
            m, edges = get_graph(code)
            
            input_data = f'{n} {m}\n'
            for v in edges:
                for u in edges[v]:
                    input_data += f'{v} {u}\n'
                    
            input_data_stupid = input_data + 'stupid\n'
            input_data_fast = input_data + 'fast\n'
            
            if i >= 494:
                draw_graph(n, edges)
                print(input_data)
            
            stdout_stupid = get_output(input_data_stupid)
            stdout_fast = get_output(input_data_fast)
            
            if stdout_fast[0] != stdout_stupid[0]:
                print(f'Error: fast = {stdout_fast[0]}, stupid = {stdout_stupid[0]}')
                draw_graph(n, edges)
                print('Data:', input_data)
                print('Edges:', dict(edges), '\n\n')
                error = True
            
    if not error:
        print('OK!')
    else:
        print('ERROR!')
            

test_graphs(7)

0it [00:00, ?it/s]

KeyboardInterrupt: 